In [1]:
# import data and functions module
from functions import *
path_308 = 'data/308.15.txt'
path_358 = 'data/358.15.txt'
path_408 = 'data/408.15.txt'

data_308 = pd.read_csv(path_308, delimiter='\t')
data_358 = pd.read_csv(path_358, delimiter='\t')
data_408 = pd.read_csv(path_408, delimiter='\t')

Q2. Table 1. molar volume, compressibility factor for CO 2 at 35o C, 85o C and 135o C over 0.18-18 MPa


In [ ]:
# Define pressure and temperature values as NumPy arrays
P_vals = 1000000*np.array([0.18, 2, 4, 6, 8, 10, 12, 14, 16, 18]) # Pa
T_vals = np.array([308.15, 358.15, 408.15]) # K
datasets = [data_308, data_358, data_408]

Table1 = pd.DataFrame(columns=["P (MPa)", "v(m3/mol)", "Z", "PR-Z", "% Diff Z"])
i = 0 # index for datasets
j = 0 # index for NIST data
k = 0 # index for Table1

for T in T_vals:
  for P in P_vals:
    Table1.at[k, "P (MPa)"] = P/1000000

    PRZ = calc_PRZ(T, P) # Calculate PR-Z using Peng-Robinson EOS
    Table1.at[k, "PR-Z"] = PRZ

    data = datasets[i]
    Z = calc_Z(T, P, data["Volume (m3/mol)"][j]) # Calculate Z using NIST data
    Table1.at[k, "v(m3/mol)"] = data["Volume (m3/mol)"][j]
    Table1.at[k, "Z"] = Z

    percent_diffz = percent_diff(Z,PRZ) # Calculate percent difference between PR-Z and NIST Z
    Table1.at[k, "% Diff Z"] = percent_diffz

    if j < 9:
      j += 1
    k += 1  
  j = 0
  i += 1

Q2. Table 2. Enthalpy difference for CO2 at specified P and T

In [ ]:
P2_vals = 1000000*np.array([2, 8, 16])
Table2 = pd.DataFrame(columns=["P (MPa)", "Delta H T85-T35 (kJ/mol)", "Delta Hdep T85-T35 (kJ/mol)", "Delta H T135-T35 (kJ/mol)", 
                               "Delta Hdep T135-T35 (kJ/mol)"])

l=0 # index for Table2
m = [1, 4, 8] # index in data, starting at P=2 MPa

for P in P2_vals: 
    Table2.at[l, "P (MPa)"] = P/1000000
    DeltaH1 = data_358["Enthalpy (kJ/mol)"][m[l]] - data_308["Enthalpy (kJ/mol)"][m[l]]
    Table2.at[l, "Delta H T85-T35 (kJ/mol)"] = DeltaH1

    DeltaHid1 = calc_idealH(308.15, 358.15)
    Hres308 = res_enth(308.15, P, calc_PRZ(308.15, P))
    Hres358 = res_enth(358.15, P, calc_PRZ(358.15, P))
    DeltaHdep1 = DeltaHid1 + (Hres358 - Hres308)
    Table2.at[l, "Delta Hdep T85-T35 (kJ/mol)"] = DeltaHdep1

    DeltaH2 = data_408["Enthalpy (kJ/mol)"][m[l]] - data_308["Enthalpy (kJ/mol)"][m[l]]
    Table2.at[l, "Delta H T135-T35 (kJ/mol)"] = DeltaH2

    DeltaHid2 = calc_idealH(308.15, 408.15)
    Hres408 = res_enth(408.15, P, calc_PRZ(408.15, P))
    DeltaHdep2 = DeltaHid2 + (Hres408 - Hres308)
    Table2.at[l, "Delta Hdep T135-T35 (kJ/mol)"] = DeltaHdep2

    l += 1

Q5. Table 3: Welec done at each stage of compression for CO2

Q6. Table 4: Heat duty of inter-stage coolers using dep function expression and ideal gas heat capacity (Cp)

In [ ]:
const_P = [5.4, 16.2, 18] # MPa
inlet_T = [412.24, 358.15, 408.15] # K
t = 0 # index for inlet_T
table4 = pd.DataFrame(columns=["Inlet P (MPa)", "Outlet P (MPa)", "Inlet T (oC)", "Outlet T (oC)", "Q (MW)"])
for P in const_P:

    table4.at[t, "Inlet P (MPa)"] = P
    table4.at[t, "Outlet P (MPa)"] = P
    table4.at[t, "Inlet T (oC)"] = inlet_T[t] - 273.15
    table4.at[t, "Outlet T (oC)"] = 40
    Q = calc_Q(inlet_T[t], 40+273.15, P*1000000, calc_mol_flow())
    table4.at[t, "Q (MW)"] = Q
    t += 1

In [4]:
Hout = res_enth(313.15, 5.4e6, calc_PRZ(313.15, 5.4e6))
Hin = res_enth(412.24, 5.4e6, calc_PRZ(412.24, 5.4e6))
Hid = calc_idealH(412.24, 313.15)
Hdep = Hid + (Hout-Hin)
print(Hdep*calc_mol_flow())

-563.3017564228367
